# URBN PL 213 - Final Project: Create Dataset
### Yu-Chen Chu
### Kongpob Leemingsawat
### 5/23/2023

# Library

In [1]:
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd 
import pandas as pd 
import numpy as np 
import cenpy
from cenpy import products
import matplotlib.pyplot as plt
import json
import requests
from IPython.display import display

# Directory

1. Import Dataset (to Merge)
- EV Charging Station Dataset (Source: DataRequest)
- Calenviroscreen Dataset (Source: [Document](https://oehha.ca.gov/calenviroscreen))
- ACS Dataset (Y:2019) - Median Household Income
- Highways Dataset (Source: [Document](https://gisdata-caltrans.opendata.arcgis.com/datasets/1f71fa512e824ff09d4b9c3f48b6d602_0/explore?location=33.953319%2C-117.793756%2C9.93))
- All Road Dataset (Source: [Document](https://catalog.data.gov/dataset/tiger-line-shapefile-2018-county-los-angeles-county-ca-all-roads-county-based-shapefile))

2. Merge Dataset

3. Create Additional Columns for ML Model

4. Drop Duplicates + Export Dataset

# 1. Import Dataset (to Merge)

### EV Charging Station Dataset

In [6]:
# import data
ev_df = pd.read_csv('Data/station_dataset.csv')

# make zip column == int (ex. from 90024.0 to 90024)
ev_df['zip'] = ev_df['zip'].astype(int)

# make id column == int
ev_df['id'] = ev_df['id'].astype(int)

# we don't need column = unnamed:0
ev_df = ev_df.drop('Unnamed: 0', axis=1)

# create a geodata frame 
ev_gdf = gpd.GeoDataFrame(ev_df, geometry = gpd.points_from_xy(ev_df.longitude, ev_df.latitude, 
                                                                crs = 'EPSG: 4326'))
# There are two latitude that seem to be off - let's drop it  
ev_gdf = ev_gdf[(ev_gdf["latitude"]>0) & (ev_gdf["latitude"] < 36)]

ev_gdf.head(2)

,id,station_name,open_date,status_code,fuel_type_code,ev_connector_types,ev_network,groups_with_access_code,access_days_time,street_address,city,zip,state,longitude,latitude,geometry
0,117709,Bexco Enterprises,2019-02-07,E,ELEC,['J1772'],Blink Network,Public,NaN,8700 Rex Rd,Pico Rivera,90660,CA,-118.105339,33.979264,POINT (-118.10534 33.97926)
1,185373,"Walmart 2886 (Pico Rivera, CA)",2021-02-25,E,ELEC,"['CHADEMO', 'J1772COMBO']",Electrify America,Public,24 hours daily,8500 WASHINGTON BLVD,Pico Rivera,90660,CA,-118.104950,33.986450,POINT (-118.10495 33.98645)


### Calenviroscreen Dataset

In [28]:
# import data - Calenviroscreen
calenviroscreen = gpd.read_file("Data/CalEnviroScreen/CES4 Final Shapefile.shp")

# filter the data to cover only Los Angeles
calenviroscreen_LA = calenviroscreen[calenviroscreen['County'] == 'Los Angeles']

# modify Tract column 
calenviroscreen_LA.Tract = calenviroscreen_LA.Tract.astype(int).astype(str).str.zfill(11).astype(int)

calenviroscreen_LA.head(2)

,Tract,ZIP,County,ApproxLoc,TotPop19,CIscore,CIscoreP,Ozone,OzoneP,PM2_5,...,Elderly65,Hispanic,White,AfricanAm,NativeAm,OtherMult,Shape_Leng,Shape_Area,AAPI,geometry
5692,6037920336,91321,Los Angeles,Santa Clarita,6881,37.794504,71.747353,0.065915,95.270691,9.600230,...,4.6214,76.5586,15.2594,0.3779,0.0,4.2581,6695.407818,1.656762e+06,3.5460,"POLYGON ((135258.650 -402745.383, 135455.130 -..."
5693,6037920044,91350,Los Angeles,Santa Clarita,2399,12.056400,18.116490,0.064647,93.627878,10.007159,...,10.7545,33.4306,49.6040,1.0004,0.0,4.4185,4568.318508,9.978606e+05,11.5465,"POLYGON ((137014.367 -395334.286, 137043.791 -..."


### ACS Median Household Income Dataset - 2019

In [9]:
# import dataset
census_df = pd.read_csv("Data/cs_inc_2019.csv")

# we don't need column = unnamed:0
census_df = census_df.drop('Unnamed: 0', axis=1)

# rename
census_df = census_df.rename(columns={'GEOID': 'Tract',
                                    'B19019_001E': 'Median HH Income'})

census_df.head(5)

,Tract,geometry,Median HH Income,NAME,state,county,tract
0,6037113235,"POLYGON ((-13210029.38 4061128.27, -13209711.0...",89792.0,"Census Tract 1132.35, Los Angeles County, Cali...",6,37,113235
1,6037920104,"POLYGON ((-13227083.75 4118563.84, -13227063.8...",127625.0,"Census Tract 9201.04, Los Angeles County, Cali...",6,37,920104
2,6037113231,"POLYGON ((-13210036.18 4059258.03, -13210036.0...",112500.0,"Census Tract 1132.31, Los Angeles County, Cali...",6,37,113231
3,6037137201,"POLYGON ((-13205088.58 4052494.87, -13205082.4...",77146.0,"Census Tract 1372.01, Los Angeles County, Cali...",6,37,137201
4,6037137401,"POLYGON ((-13204709.76 4050354.03, -13204603.3...",125781.0,"Census Tract 1374.01, Los Angeles County, Cali...",6,37,137401


### Highways Dataset

In [12]:
# import dataset 
hw_gdf = gpd.read_file("Data/National_Highway_System.geojson")

# filter LA county
LA_hw_gdf = hw_gdf[hw_gdf['County'] == 'Los Angeles']

LA_hw_gdf.head()

,OBJECTID,EventID,RouteID,FromARMeasure,ToARMeasure,NHS_TYPE,County,City,Shape_Length,geometry
13,14,8835,SHS_001._P,53.691452,59.975394,M21PA,Los Angeles,Los Angeles,0.090427,MULTILINESTRING Z ((-118.45276 33.99412 0.0000...
14,15,8835,SHS_001._P,53.691452,59.975394,M21PA,Los Angeles,Santa Monica,0.000553,MULTILINESTRING Z ((-118.47011 34.00248 0.0000...
15,16,8835,SHS_001._P,53.691452,59.975394,M21PA,Los Angeles,County,0.006657,MULTILINESTRING Z ((-118.44149 33.98321 0.0000...
16,17,8835,SHS_001._P,59.976394,60.023533,M21PA,Los Angeles,Santa Monica,0.000769,MULTILINESTRING Z ((-118.48576 34.01478 0.0000...
17,18,8835,SHS_001._P,60.024533,83.930672,M21PA,Los Angeles,Malibu,0.281487,MULTILINESTRING Z ((-118.59519 34.03997 0.0000...


### All-Roads Dataset

In [14]:
LAroads = gpd.read_file("Data/LA_roads.zip")
LAroads.head()

,LINEARID,FULLNAME,RTTYP,MTFCC,geometry
0,1101576755652,Golden State Fwy Rmp,M,S1400,"LINESTRING (-118.24370 34.09428, -118.24456 34..."
1,1101576692583,Pomona Fwy Rmp,M,S1400,"LINESTRING (-118.16925 34.03696, -118.16949 34..."
2,1101576663753,Soto St Rmp,M,S1400,"LINESTRING (-118.19800 34.06246, -118.19786 34..."
3,1101576715256,Ventura Fwy Rmp,M,S1400,"LINESTRING (-118.63728 34.16101, -118.63740 34..."
4,1101576716867,Almansor St Ovp,M,S1400,"LINESTRING (-118.11680 34.07039, -118.11682 34..."


# 2. Merge Dataset I

### ACS + Calenviroscreen = joinedDf_1

In [38]:
joinedDf_1 = pd.merge(calenviroscreen_LA, census_df[['Tract','Median HH Income']],
                         how = 'left', on = 'Tract')
joinedDf_1.head(2)

,Tract,ZIP,County,ApproxLoc,TotPop19,CIscore,CIscoreP,Ozone,OzoneP,PM2_5,...,Hispanic,White,AfricanAm,NativeAm,OtherMult,Shape_Leng,Shape_Area,AAPI,geometry,Median HH Income
0,6037920336,91321,Los Angeles,Santa Clarita,6881,37.794504,71.747353,0.065915,95.270691,9.600230,...,76.5586,15.2594,0.3779,0.0,4.2581,6695.407818,1.656762e+06,3.5460,"POLYGON ((135258.650 -402745.383, 135455.130 -...",56912.0
1,6037920044,91350,Los Angeles,Santa Clarita,2399,12.056400,18.116490,0.064647,93.627878,10.007159,...,33.4306,49.6040,1.0004,0.0,4.4185,4568.318508,9.978606e+05,11.5465,"POLYGON ((137014.367 -395334.286, 137043.791 -...",98472.0


### joinedDF_1 + EV Dataset = joinedDf_2

In [140]:
# join two geodataframe
joinedDf_2 = gpd.sjoin(ev_gdf, joinedDf_1.to_crs("EPSG:4326"),
                         how = 'inner', op = 'intersects')

joinedDf_2.head(2)

,id,station_name,open_date,status_code,fuel_type_code,ev_connector_types,ev_network,groups_with_access_code,access_days_time,street_address,...,Elderly65,Hispanic,White,AfricanAm,NativeAm,OtherMult,Shape_Leng,Shape_Area,AAPI,Median HH Income
0,117709,Bexco Enterprises,2019-02-07,E,ELEC,['J1772'],Blink Network,Public,NaN,8700 Rex Rd,...,14.6701,94.1741,4.4689,0.2808,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0
1,185373,"Walmart 2886 (Pico Rivera, CA)",2021-02-25,E,ELEC,"['CHADEMO', 'J1772COMBO']",Electrify America,Public,24 hours daily,8500 WASHINGTON BLVD,...,14.6701,94.1741,4.4689,0.2808,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0


# 3. Create Additional Columns for ML Model

In [141]:
# 1. Number of EV charging station by tract
n_ev = joinedDf_2.groupby('Tract').size()
n_ev.name = 'n_ev'

# merge back to joinedDf_2
joinedDf_2 = joinedDf_2.merge(n_ev, how = 'left', on = 'Tract')
joinedDf_2 = joinedDf_2.fillna(0)

joinedDf_2.head(2)

,id,station_name,open_date,status_code,fuel_type_code,ev_connector_types,ev_network,groups_with_access_code,access_days_time,street_address,...,Hispanic,White,AfricanAm,NativeAm,OtherMult,Shape_Leng,Shape_Area,AAPI,Median HH Income,n_ev
0,117709,Bexco Enterprises,2019-02-07,E,ELEC,['J1772'],Blink Network,Public,0,8700 Rex Rd,...,94.1741,4.4689,0.2808,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3
1,185373,"Walmart 2886 (Pico Rivera, CA)",2021-02-25,E,ELEC,"['CHADEMO', 'J1772COMBO']",Electrify America,Public,24 hours daily,8500 WASHINGTON BLVD,...,94.1741,4.4689,0.2808,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3


In [142]:
# 2. Number of road by tract 

# To do so, let's merge road dataset with calenviroscreen dataset 
LAroads.to_crs('EPSG:3857', inplace = True)
calenviroscreen_LA.to_crs('EPSG:3857', inplace = True)
joinedDf_3 = gpd.sjoin(LAroads, calenviroscreen_LA[['Tract','geometry']], how='inner', predicate='intersects')

# count by tract 
n_roads = joinedDf_3.groupby('Tract').size()
n_roads.name = 'n_roads'

# merge back to joinedDf_2
joinedDf_2 = joinedDf_2.merge(n_roads, how = 'left', on = 'Tract')
joinedDf_2.head(2)

,id,station_name,open_date,status_code,fuel_type_code,ev_connector_types,ev_network,groups_with_access_code,access_days_time,street_address,...,White,AfricanAm,NativeAm,OtherMult,Shape_Leng,Shape_Area,AAPI,Median HH Income,n_ev,n_roads
0,117709,Bexco Enterprises,2019-02-07,E,ELEC,['J1772'],Blink Network,Public,0,8700 Rex Rd,...,4.4689,0.2808,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3,113
1,185373,"Walmart 2886 (Pico Rivera, CA)",2021-02-25,E,ELEC,"['CHADEMO', 'J1772COMBO']",Electrify America,Public,24 hours daily,8500 WASHINGTON BLVD,...,4.4689,0.2808,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3,113


In [143]:
# 3. Number of freeway by tract 

# To do so, let's merge freeway dataset with calenviroscreen dataset 
LA_hw_gdf.to_crs('EPSG:3857', inplace = True)
joinedDf_4 = gpd.sjoin(LA_hw_gdf, calenviroscreen_LA[['Tract','geometry']], how='inner', predicate='intersects')

# count by tract 
n_hws = joinedDf_4.groupby('Tract').size()
n_hws.name = 'n_highways'

# merge back to joinedDf_2
joinedDf_2 = joinedDf_2.merge(n_hws, how = 'left', on = 'Tract')
joinedDf_2.head(2)

,id,station_name,open_date,status_code,fuel_type_code,ev_connector_types,ev_network,groups_with_access_code,access_days_time,street_address,...,AfricanAm,NativeAm,OtherMult,Shape_Leng,Shape_Area,AAPI,Median HH Income,n_ev,n_roads,n_highways
0,117709,Bexco Enterprises,2019-02-07,E,ELEC,['J1772'],Blink Network,Public,0,8700 Rex Rd,...,0.2808,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3,113,12.0
1,185373,"Walmart 2886 (Pico Rivera, CA)",2021-02-25,E,ELEC,"['CHADEMO', 'J1772COMBO']",Electrify America,Public,24 hours daily,8500 WASHINGTON BLVD,...,0.2808,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3,113,12.0


In [144]:
# 4. Distance to road 

# set geometry
joinedDf_2 = joinedDf_2.set_geometry('geometry')

# get distance to road for each station
temp = gpd.sjoin_nearest(joinedDf_2.to_crs('EPSG:3857'), # reset the crs to 3857 to make the unit 'meter' 
                           LAroads.to_crs('EPSG:3857'),
                           how = 'left',
                           lsuffix='_EV',
                           rsuffix='_Road',
                           distance_col='dist_to_rd')

# merge back to joinedDf_2
joinedDf_2 = pd.merge(joinedDf_2, temp[['id', 'dist_to_rd']], how = 'left', on = 'id')
joinedDf_2.head(2)

,id,station_name,open_date,status_code,fuel_type_code,ev_connector_types,ev_network,groups_with_access_code,access_days_time,street_address,...,NativeAm,OtherMult,Shape_Leng,Shape_Area,AAPI,Median HH Income,n_ev,n_roads,n_highways,dist_to_rd
0,117709,Bexco Enterprises,2019-02-07,E,ELEC,['J1772'],Blink Network,Public,0,8700 Rex Rd,...,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3,113,12.0,33.453430
1,185373,"Walmart 2886 (Pico Rivera, CA)",2021-02-25,E,ELEC,"['CHADEMO', 'J1772COMBO']",Electrify America,Public,24 hours daily,8500 WASHINGTON BLVD,...,0.0,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3,113,12.0,87.711045


In [145]:
# 5. Distance to highway 

# get distance to highway for each station 
temp = gpd.sjoin_nearest(joinedDf_2.to_crs('EPSG:3857'), 
                           LA_hw_gdf.to_crs('EPSG:3857'),  
                           how = 'left', # reset the crs to 3857 to make the unit 'meter' 
                           lsuffix='_EV', 
                           rsuffix='_HW', 
                           distance_col='dist_to_hw')

# merge back to joinedDf_2
joinedDf_2 = pd.merge(joinedDf_2, temp[['id', 'dist_to_hw']], how = 'left', on = 'id')
joinedDf_2.head(2)

,id,station_name,open_date,status_code,fuel_type_code,ev_connector_types,ev_network,groups_with_access_code,access_days_time,street_address,...,OtherMult,Shape_Leng,Shape_Area,AAPI,Median HH Income,n_ev,n_roads,n_highways,dist_to_rd,dist_to_hw
0,117709,Bexco Enterprises,2019-02-07,E,ELEC,['J1772'],Blink Network,Public,0,8700 Rex Rd,...,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3,113,12.0,33.453430,492.54981
1,185373,"Walmart 2886 (Pico Rivera, CA)",2021-02-25,E,ELEC,"['CHADEMO', 'J1772COMBO']",Electrify America,Public,24 hours daily,8500 WASHINGTON BLVD,...,0.0,8633.071338,3.825265e+06,1.0763,63098.0,3,113,12.0,87.711045,89.20828


In [146]:
# 6. Number of Roads by Buffer around EV Charging Stations 

# let's use the temp dataset from abpve to perform this task 
# Create 0.5 mile buffer for EV stations (0.5 mile = 804.67 meters)

temp['buffer_0.5'] = temp.geometry.buffer(804.67) 

# Set 'buffer_0.5' as the active geometry column
temp = temp.set_geometry('buffer_0.5')

# Spatially join temp and LAroads dataset 
temp2 = gpd.sjoin(temp[['id','buffer_0.5']].to_crs('EPSG:3857'), # The unit of 'EPSG:3857' is meter while 'EPSG:4326'is degree
                   LAroads.to_crs('EPSG:3857'),
                   how='left',
                   predicate='intersects')

# Groupby the number of roads by buffer
n_rd_buffer = temp2.groupby('id').size()
n_rd_buffer.name = 'n_rd_buffer'

# merge back to joinedDf_2
joinedDf_2 = joinedDf_2.merge(n_rd_buffer, how = 'left', on = 'id')
joinedDf_2.head(2)

,id,station_name,open_date,status_code,fuel_type_code,ev_connector_types,ev_network,groups_with_access_code,access_days_time,street_address,...,Shape_Leng,Shape_Area,AAPI,Median HH Income,n_ev,n_roads,n_highways,dist_to_rd,dist_to_hw,n_rd_buffer
0,117709,Bexco Enterprises,2019-02-07,E,ELEC,['J1772'],Blink Network,Public,0,8700 Rex Rd,...,8633.071338,3.825265e+06,1.0763,63098.0,3,113,12.0,33.453430,492.54981,54
1,185373,"Walmart 2886 (Pico Rivera, CA)",2021-02-25,E,ELEC,"['CHADEMO', 'J1772COMBO']",Electrify America,Public,24 hours daily,8500 WASHINGTON BLVD,...,8633.071338,3.825265e+06,1.0763,63098.0,3,113,12.0,87.711045,89.20828,46


In [147]:
# 7. Number of Highways by Buffer around EV Charging Stations

# let's use the temp dataset from above to perform this task 

# Spatially join EVgdf and highway df
temp2 = gpd.sjoin(temp[['id','buffer_0.5']].to_crs('EPSG:3857'), # The unit of 'EPSG:3857' is meter while 'EPSG:4326'is degree
                   LA_hw_gdf.to_crs('EPSG:3857'),
                   how='inner',
                   predicate='intersects')

# Groupby the number of highways by buffer
n_hw_buffer = temp2.groupby('id').size()
n_hw_buffer.name = 'n_hw_buffer'

# merge back to joinedDf_2
joinedDf_2 = joinedDf_2.merge(n_hw_buffer, how = 'left', on = 'id')
joinedDf_2.head(2)

,id,station_name,open_date,status_code,fuel_type_code,ev_connector_types,ev_network,groups_with_access_code,access_days_time,street_address,...,Shape_Area,AAPI,Median HH Income,n_ev,n_roads,n_highways,dist_to_rd,dist_to_hw,n_rd_buffer,n_hw_buffer
0,117709,Bexco Enterprises,2019-02-07,E,ELEC,['J1772'],Blink Network,Public,0,8700 Rex Rd,...,3.825265e+06,1.0763,63098.0,3,113,12.0,33.453430,492.54981,54,2.0
1,185373,"Walmart 2886 (Pico Rivera, CA)",2021-02-25,E,ELEC,"['CHADEMO', 'J1772COMBO']",Electrify America,Public,24 hours daily,8500 WASHINGTON BLVD,...,3.825265e+06,1.0763,63098.0,3,113,12.0,87.711045,89.20828,46,4.0


# 4. Drop Duplicates + Export Dataset

In [148]:
# copy the joinedDf_2 
complete_dataset_ev = joinedDf_2.copy()

# set-up the index 
complete_dataset_ev.set_index('id', inplace = True)

# check the duplicated index 
duplicate_rows = complete_dataset_ev[complete_dataset_ev.index.duplicated(keep = False)]

# Groupby index and take the first row of any duplicated data
complete_dataset_ev_unique = complete_dataset_ev.groupby('id').first()

# compare before and after
print("Before :", complete_dataset_ev.shape)
print("After :", complete_dataset_ev_unique.shape)

Before : (4844, 90)
After : (3828, 90)


In [151]:
# Export dataset 

complete_dataset_ev_unique.reset_index(inplace = True)
complete_dataset_ev_unique.to_csv(r'Data/completed_dataset.csv')